# Elixir: Erlang libraries

Elixir provides excellent interoperability with Erlang libraries. In fact, Elixir discourages simply wrapping Erlang libraries in favor of directly interfacing with Erlang code. This section describes some common Erlang functionality that is not found in Elixir.

Consider exploring the [Erlang STDLIB Reference Manual](http://erlang.org/doc/apps/stdlib/index.html).

### the [binary module](http://erlang.org/doc/man/binary.html)

The built-in Elixir String module handles binaries that are UTF-8 encoded. The binary module is useful when you are dealing with binary data that is __not__ UTF-8 encoded.

Here, the `String` module returns Unicode codepoints, while `:binary` deals with raw data bytes.

In [1]:
String.to_charlist "Ø"

[216]

In [2]:
:binary.bin_to_list "Ø"

[195, 152]

### [Formatted text output](http://erlang.org/doc/man/io.html#format-1)

Elixir does not contain a function similar to `printf` found in C and other languages. Erlang provides the standard library functions `:io.format/2` and `:io_lib.format/2` instead. The first formats to terminal output, while the second formats to an iolist.

In [3]:
:io.format("Pi is approximately given by:~10.3f~n", [:math.pi])

Pi is approximately given by:     3.142


:ok

In [4]:
to_string :io_lib.format("Pi is approximately given by:~10.3f~n", [:math.pi])

"Pi is approximately given by:     3.142\n"

### the [crypto module](http://erlang.org/doc/man/crypto.html)

This module contains hashing functions, digital signatures, encryption and more. 

In [5]:
Base.encode16(:crypto.hash(:sha256, "Elixir"))

"3315715A7A3AD57428298676C5AE465DADA38D951BDFAC9348A8A31E9C7401CB"

It is not part of the Erlang standard library, but is included with the Erlang distribution. This means you must list `:crypto` in your project’s applications list to use it. To do this, edit your `mix.exs` file to include:

`def application do
  [extra_applications: [:crypto]]
end`

### the [digraph module](http://erlang.org/doc/man/digraph.html) and [digraph_utils](http://erlang.org/doc/man/digraph_utils.html)

`digraph` and `digraph_utils` contain functions for handling directed graphs built of vertices and edges. After constructing the graph, the algorithms will help find, for instance, the shortest path between two vertices, or loops in the graph.

Given three vertices, find the shortest path from the first to the last.

In [6]:
digraph = :digraph.new()

{:digraph, #Reference<0.943104496.2083127297.86895>, #Reference<0.943104496.2083127297.86896>, #Reference<0.943104496.2083127297.86897>, true}

In [7]:
coords = [{0.0, 0.0}, {1.0, 0.0}, {1.0, 1.0}]

[{0.0, 0.0}, {1.0, 0.0}, {1.0, 1.0}]

In [8]:
[v0, v1, v2] = (for c <- coords, do: :digraph.add_vertex(digraph, c))

[{0.0, 0.0}, {1.0, 0.0}, {1.0, 1.0}]

In [9]:
:digraph.add_edge(digraph, v0, v1)
:digraph.add_edge(digraph, v1, v2)
:digraph.get_short_path(digraph, v0, v2)

[{0.0, 0.0}, {1.0, 0.0}, {1.0, 1.0}]

Note: `:digraph` functions alter the graph structure in-place. This is possible because they are implemented as ETS tables (explained next).

### Erlang Term Storage (ETS)

The [ets](http://erlang.org/doc/man/ets.html) and [dets](http://erlang.org/doc/man/dets.html) modules handle storage of large data structures in memory or on disk.

ETS lets you create a table containing tuples. ETS tables are protected by default - only the owner process may write to the table, but any other process can read. 

ETS can allow a table to be used as a simple database, a key-value store or as a cache mechanism.

`ets` module functions will modify the state of the table as a side-effect.

__NOTE: :ets.i(table) appears to wait for for user input before exiting. Needs further investigation.__

![table](pix/table.png)

### the [math module](http://erlang.org/doc/man/math.html)

This module covers trigonometry, exponential & logarithmic functions.

In [10]:
angle_45_deg = :math.pi() * 45.0 / 180.0

0.7853981633974483

In [11]:
:math.sin(angle_45_deg)

0.7071067811865475

In [12]:
:math.exp(55.0)

7.694785265142018e23

In [14]:
:math.log(7.69478e23)

54.99999931575169

### the [queue module](http://erlang.org/doc/man/queue.html)

This module implements double-ended FIFO queues.

In [15]:
q = :queue.new

{[], []}

In [16]:
q = :queue.in("A", q)
q = :queue.in("B", q)
{value, q} = :queue.out(q)

{{:value, "A"}, {[], ["B"]}}

In [17]:
value

{:value, "A"}

In [18]:
{value, q} = :queue.out(q)
value

{:value, "B"}

In [19]:
{value, q} = :queue.out(q)
value

:empty

### the [rand module](http://erlang.org/doc/man/rand.html)

This module returns random values and sets the random seed.

In [20]:
:rand.uniform()

0.8757444731543029

In [21]:
:rand.seed(:exs1024, {123,123534,3345345})

{%{jump: #Function<13.10897371/1 in :rand.mk_alg/1>, max: 18446744073709551615, next: #Function<12.10897371/1 in :rand.mk_alg/1>, type: :exs1024}, {[6553152330993393869, 15576460181258137320, 2960463442740161846, 13247468477590660091, 3207070858802105896, 679965480797542391, 10180771202394949538, 1801809308031995977, 4023298332664308366, 1363327370850698446, 7095173100638256885, 13137639840664349999, 14412257979290585054, 10860747010201920933, 3676145946860864412, 5490211022645046085], []}}

In [22]:
:rand.uniform()

0.7404221885067003

In [23]:
:rand.uniform(6)

5

In [24]:
:rand.uniform(6)

3

### the [zip module](http://erlang.org/doc/man/rand.html)

This module  lets you read/write ZIP files to/from disk/memory, and extract file information.

In [25]:
song = "
Mary had a little lamb,
His fleece was white as snow,
And everywhere that Mary went,
The lamb was sure to go."

"\nMary had a little lamb,\nHis fleece was white as snow,\nAnd everywhere that Mary went,\nThe lamb was sure to go."

In [26]:
compressed = :zlib.compress(song)

<<120, 156, 37, 140, 187, 13, 195, 48, 12, 5, 123, 77, 241, 6, 16, 188, 67, 186, 52, 233, 188, 0, 109, 189, 132, 2, 100, 9, 144, 152, 16, 222, 62, 254, 116, 87, 220, 93, 120, 73, 223, 161, 146, 32, 40, 217, 172, 16, 69, 182, ...>>

In [27]:
byte_size song

110

In [28]:
byte_size compressed

99

In [29]:
:zlib.uncompress(compressed)

"\nMary had a little lamb,\nHis fleece was white as snow,\nAnd everywhere that Mary went,\nThe lamb was sure to go."